## Dealing with Missing Data


### When reading in missing values, pandas will display them as NaN values.
### There are also newer specialized null pandas values such as pd.NaT to imply the value missing should be a timestamp.

Options for missing Data:
-  1. Keep them
-  2. Remove them
-   3. Replace them with a value
-   4. Replace them with a statistic (mean, median, mode)
-   5. Use interpolation to estimate the missing values
-   6. Use machine learning models to predict the missing values
-   7. Use domain knowledge to fill in the missing values

Keeping a missing data:
* Pros:
  - Easiest to do
  - Does not manipulate or change the true data
* Cons:
  - Many methods do not support NaN
  - Often there are reasonable guesses

Dropping or removing the missing data:(Makes sense when a lot of data is missing)
* Pros: 
  - Easy to do
  - Can do based on rules
* Cons:
  - Potential to lose a lot of dat or useful information
  - Limits trained models for future data.

(Often a good idea to calculate the percent of data which is missing)


Filling the missing data:
* Pros:
  - Potential to save a lot of data for use in training a model
* Cons:
  - Hardest to do and somewhat arbitrary
  - Potential to lead to false conclusions

* Filling the same value => good choice if NaN was a placeholder
* Filling with interpolated or estimated value => much harder and requires reasonable assumptions



In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('4_tips.csv')